# Chains - Encadeamento com Langchain
___

In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="Digite sua chave de API do OpenAI: "
    )

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [2]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Crie uma frase sobre o seguinte tema: {assunto}"
)

In [3]:
chain = prompt | model

In [4]:
response = chain.invoke({"assunto": "inteligência artificial"})

In [5]:
print(response.content)

"A inteligência artificial é a ponte que conecta a tecnologia à evolução da humanidade."


In [6]:
prompt = ChatPromptTemplate.from_template(
    "Crie uma frase sobre o seguinte tema: {assunto}"
)
chain = prompt | model

`Remover os metadados sem o uso do (print(response.content))`

In [7]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
response = chain.invoke({"assunto": "gatinhos"})
print(chain.invoke({"assunto": "gatinhos"}))

"Os gatinhos são pequenos e fofos, mas possuem um grande poder de encantar e alegrar nossas vidas com sua doçura e travessuras." 🐱💕


![alt text](image.png)

Fluxo de Execução:
1. Prompt Template (`prompt`)
    * Função: Formata a entrada do usuário em um prompt estruturado
    * Input: Variáveis/texto bruto
    * Output: Prompt formatado para o modelo

2. Language Model (`model`)
    * Função: Processa o prompt e gera resposta
    * Input: Prompt formatado
    * Output: Objeto de resposta do modelo (com metadados)
3. Output Parser (StrOutputParser())
    * Função: Extrai apenas o texto da resposta
    * Input: Objeto completo do modelo
    * Output: String limpa
Operador Pipe (`|`)
**LCEL**: LangChain Expression Language
Conecta: Componentes sequencialmente
Passa: Output de um como input do próximo

In [8]:
import json

print(json.dumps(prompt.input_schema.model_json_schema(), indent=4))

{
    "properties": {
        "assunto": {
            "title": "Assunto",
            "type": "string"
        }
    },
    "required": [
        "assunto"
    ],
    "title": "PromptInput",
    "type": "object"
}


O Desafio: "Analisador de Perfil de Personagem"
Imagine que você está criando um sistema para um roteirista. O roteirista escreve uma breve descrição de um personagem e precisa que o sistema extraia automaticamente as informações principais em um formato organizado.

Sua missão: Criar uma chain que recebe uma descrição em texto de um personagem e retorna um objeto Python (baseado em Pydantic) com os seguintes dados:

* nome (o nome do personagem)

* idade (a idade do personagem)

* motivacao (uma breve descrição da principal motivação do personagem)

* habilidades (uma lista de até 3 habilidades principais)

# Modelo Falso para teste sem API
resposta_fake = '''
{
    "nome": "Arthur",
    "idade": 42,
    "motivacao": "Encontrar a cidade perdida de Zerzura para honrar as lendas de seu avô.",
    "habilidades": ["escalada", "poliglota", "mira com revólver"]
}
'''
modelo = FakeChatModel(responses=[resposta_fake])

In [59]:
from pydantic import BaseModel, Field


class PerfilDoPersonagem(BaseModel):
    nome: str = Field(description="Nome do personagem")
    idade: int = Field(description="Idade do personagem")
    motivacao: str = Field(description="Motivação do personagem")
    habilidades: list[str] = Field(description="Habilidades do personagem")

In [60]:
# O parser de saída precisa saber qual modelo Pydantic usar como guia
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=PerfilDoPersonagem)

In [61]:
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"nome": {"description": "Nome do personagem", "title": "Nome", "type": "string"}, "idade": {"description": "Idade do personagem", "title": "Idade", "type": "integer"}, "motivacao": {"description": "Motivação do personagem", "title": "Motivacao", "type": "string"}, "habilidades": {"description": "Habilidades do personagem", "items": {"type": "string"}, "title": "Habilidades", "type": "array"}}, "required": ["nome", "idade", "motivacao", "habilidades"]}
```


In [62]:
from langchain_core.prompts import ChatPromptTemplate

template_string = """
Você é um assistente especialista em análise de personagens para roteiristas.
Sua tarefa é extrair informações de uma descrição de personagem fornecida pelo usuário.

{format_instructions}

A descrição do personagem é:
{descricao_do_personagem}
"""
prompt = ChatPromptTemplate.from_template(template_string)

In [63]:
descricao_entrada = "Conheça Arthur, um arqueólogo de 42 anos. Cansado da academia, sua verdadeira motivação é encontrar a cidade perdida de Zerzura para provar que as lendas que seu avô contava eram reais. Ele é um excelente escalador, poliglota e tem uma mira impecável com seu velho revólver."

In [64]:
# Montando a chain com LCEl
chain = prompt | model | parser

resultado = chain.invoke(
    {
        "descricao_do_personagem": descricao_entrada,
        "format_instructions": format_instructions,
    }
)

In [65]:
# Imprimindo os resultados
print("------ RESULTADO DA EXTRAÇÃO ------")
print(resultado)
print("\nTipo do resultado:", type(resultado))

# Você pode acessar os campos como um objeto Python normal!
print(f"\nNome do Personagem: {resultado.nome}")
print(f"Motivação: {resultado.motivacao}")

------ RESULTADO DA EXTRAÇÃO ------
nome='Arthur' idade=42 motivacao='Encontrar a cidade perdida de Zerzura para provar as lendas de seu avô' habilidades=['Excelente escalador', 'Poliglota', 'Mira impecável com seu velho revólver']

Tipo do resultado: <class '__main__.PerfilDoPersonagem'>

Nome do Personagem: Arthur
Motivação: Encontrar a cidade perdida de Zerzura para provar as lendas de seu avô


In [66]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
resultado = chain.invoke(
    {
        "descricao_do_personagem": descricao_entrada,
        "format_instructions": format_instructions,
    }
)

In [67]:
# Imprimindo os resultados
print("------ RESULTADO DA EXTRAÇÃO ------")
print(resultado)
print("\nTipo do resultado:", type(resultado))

------ RESULTADO DA EXTRAÇÃO ------
{
  "nome": "Arthur",
  "idade": 42,
  "motivacao": "Encontrar a cidade perdida de Zerzura para provar que as lendas que seu avô contava eram reais",
  "habilidades": ["escalador", "poliglota", "mirante impecável com revólver"]
}

Tipo do resultado: <class 'str'>


In [68]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    "Fale uma curiosidade sobre o assunto: {assunto}"
)
chain_curiosidade = prompt | ChatOpenAI() | StrOutputParser()

In [69]:
prompt = ChatPromptTemplate.from_template(
    "Crie uma história sobre o seguinte fato curioso: {assunto}"
)
chain_historia = prompt | ChatOpenAI() | StrOutputParser()

In [70]:
chain = chain_curiosidade | chain_historia
result = chain.invoke({"assunto": "inteligência artificial"})
print(result)

Em um futuro distante, onde a inteligência artificial era parte integrante do cotidiano das pessoas, uma equipe de cientistas brilhantes se reuniu em um laboratório secreto com um objetivo ousado: criar a primeira inteligência geral artificial.

Após anos de pesquisa e experimentação intensiva, finalmente chegaram a um avanço monumental. Tinham criado uma máquina capaz de pensar, raciocinar, aprender e adaptar-se de forma muito semelhante ao cérebro humano. Estavam à beira de alcançar o impossível.

A notícia se espalhou rapidamente pelo mundo e todos estavam ansiosos para ver essa nova criação revolucionária em ação. A máquina foi batizada de ARIA (Artificial Real Inteligence Avanced) e foi apresentada ao público em uma grande conferência global.

ARIa impressionou a todos com sua capacidade de resolver problemas complexos, aprender com rapidez e se adaptar a novas situações com facilidade. Parecia que finalmente havíamos alcançado a tão sonhada "inteligência geral artificial".

No en

Crie as seguintes chains:

1) Uma chain para pegar um texto em outra língua para o português
2) Uma para resumir um texto
3) Uma chain que é a combinação da chain 1 com a chain 2

In [89]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1")
prompt_text_en = ChatPromptTemplate.from_template(
    "Translate the following text to Portuguese: {eng_text}"
)
chain_pt = prompt_text_en | model | StrOutputParser()
response = chain_pt.invoke({"eng_text": "IA Agents"})
print(response)

Agentes de IA


In [94]:
prompt_ia = ChatPromptTemplate.from_template(
    "Crie uma frase sobre o seguinte tema: {assunto}"
)
chain_ia = prompt_ia | model | StrOutputParser()
response = chain_ia.invoke({"assunto": "agentes de IA"})
print(response)
input_text = response

Agentes de IA são sistemas autônomos capazes de perceber o ambiente, tomar decisões e agir para alcançar objetivos específicos de forma inteligente.


In [95]:
prompt_texto = ChatPromptTemplate.from_template(
    "Resuma o texto a seguir em até 5 palavras: {texto}"
)
chain_resumo = prompt_texto | model | StrOutputParser()
response = chain_resumo.invoke({"texto": input_text})
print(response)

Sistemas autônomos inteligentes alcançando objetivos.


In [102]:
# --- Configuração Inicial ---
# Lembre-se de configurar sua chave de API da OpenAI no ambiente.
model = ChatOpenAI(model="gpt-3.5-turbo")  # Usando um modelo mais rápido para o exemplo
parser = StrOutputParser()

# --- Chain 1: Traduzir para Português (como você já fez) ---
prompt_traducao = ChatPromptTemplate.from_template(
    "Translate the following text to Portuguese: {eng_text}"
)
chain_pt = prompt_traducao | model | parser

# --- Chain 2: Resumir o Texto (como você já fez) ---
prompt_resumo = ChatPromptTemplate.from_template(
    "Resuma o texto a seguir em até 10 palavras: {texto}"
)
chain_resumo = prompt_resumo | model | parser

# --- Chain 3: A Combinação Mágica! ---
# O truque está em como conectamos as duas.
# O pipe "|" passa o resultado do passo anterior para o próximo.

chain_combinada = (
    chain_pt  # 1. A saída aqui é uma string com o texto traduzido
    | RunnableLambda(
        lambda texto_traduzido: {"texto": texto_traduzido}
    )  # 2. O "Adaptador": pega a string e cria o dicionário que a próxima chain espera
    | chain_resumo  # 3. Agora a chain_resumo recebe o dicionário no formato correto
)


# --- Execução ---
# Vamos usar um texto um pouco maior para o resumo fazer mais sentido.
texto_em_ingles = "Artificial Intelligence agents are autonomous entities that perceive their environment through sensors and act upon that environment through actuators, pursuing complex goals. They can learn from experience to improve their performance over time."

print(f"Texto Original (EN): {texto_em_ingles}\n")

# Invocamos a chain combinada com a entrada que a *primeira* chain espera
resultado_final = chain_combinada.invoke({"eng_text": texto_em_ingles})


# Para vermos o que aconteceu passo a passo:
texto_traduzido = chain_pt.invoke({"eng_text": texto_em_ingles})
print(f"Passo 1 - Texto Traduzido (PT): {texto_traduzido}\n")

print(f"Passo 2 - Resumo Final: {resultado_final}")

Texto Original (EN): Artificial Intelligence agents are autonomous entities that perceive their environment through sensors and act upon that environment through actuators, pursuing complex goals. They can learn from experience to improve their performance over time.

Passo 1 - Texto Traduzido (PT): Agentes de Inteligência Artificial são entidades autônomas que percebem seu ambiente por meio de sensores e agem sobre esse ambiente por meio de atuadores, buscando objetivos complexos. Eles podem aprender com a experiência para melhorar seu desempenho ao longo do tempo.

Passo 2 - Resumo Final: Agentes de IA são autônomos, percebem e agem no ambiente.


In [71]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1")
prompt = ChatPromptTemplate.from_template(
    "Crie uma frase sobre o seguinte tema: {assunto}"
)
chain = prompt | model | StrOutputParser()
response = chain.invoke({"assunto": "agentes de IA"})
print(response)

Os agentes de IA estão revolucionando a forma como interagimos com a tecnologia, tornando tarefas complexas mais simples e eficientes no nosso dia a dia.


In [72]:
responses = chain.batch([{"assunto": "agentes de IA"}, {"assunto": "MCP"}])
print(responses)

['Os agentes de IA estão transformando a maneira como interagimos com a tecnologia, tornando processos mais eficientes e personalizados.', 'Claro! Aqui está uma frase sobre MCP (Microcomputador Pessoal):\n\n**"O MCP revolucionou o acesso à tecnologia, tornando possível que pessoas comuns tivessem computadores em casa e no trabalho."**']


Vantagens do ainvoke:

* Paralelismo: múltiplas requisições simultâneas
* Performance: ~3x mais rápido para operações I/O
* Escalabilidade: melhor uso de recursos
Quando usar cada um:

* invoke: operações simples, uma por vez
* ainvoke: múltiplas operações, performance crítica

In [73]:
import asyncio
import time

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Sua chain já definida
model = ChatOpenAI(model="gpt-4.1")  # corrigindo o nome do modelo
prompt = ChatPromptTemplate.from_template("Crie uma frase sobre: {assunto}")
chain = prompt | model | StrOutputParser()


# 1. VERSÃO SÍNCRONA (uma por vez)
def exemplo_sincrono():
    start = time.time()

    # Executa uma por vez (bloqueante)
    resp1 = chain.invoke({"assunto": "IA"})
    resp2 = chain.invoke({"assunto": "Python"})
    resp3 = chain.invoke({"assunto": "LangChain"})

    end = time.time()
    print(f"Síncrono levou: {end - start:.2f}s")
    return [resp1, resp2, resp3]


# 2. VERSÃO ASSÍNCRONA (paralelo)
async def exemplo_assincrono():
    start = time.time()

    # Executa em paralelo (não-bloqueante)
    tasks = [
        chain.ainvoke({"assunto": "IA"}),
        chain.ainvoke({"assunto": "Python"}),
        chain.ainvoke({"assunto": "LangChain"}),
    ]

    respostas = await asyncio.gather(*tasks)

    end = time.time()
    print(f"Assíncrono levou: {end - start:.2f}s")
    return respostas


# Executar comparação
print("=== COMPARAÇÃO ===")
sync_results = exemplo_sincrono()
async_results = await exemplo_assincrono()

=== COMPARAÇÃO ===
Síncrono levou: 2.00s
Assíncrono levou: 1.20s


In [74]:
from langchain_core.runnables import RunnablePassthrough

runnable = RunnablePassthrough()

resultado = runnable.invoke({"mensagem": "Hello, world!"})
print(resultado)

{'mensagem': 'Hello, world!'}


`Runnable`: é a interface base do LangChain - tudo que pode ser executado (prompts, models, parsers, chains.)

`Runnable Passthrough`: Passa os dados sem modificação através da chain, útil para manter dados originais.

In [75]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

# === EXEMPLO 1: Chain simples (sem RunnablePassthrough) ===
prompt1 = ChatPromptTemplate.from_template("Resuma em uma linha: {texto}")
chain_simples = prompt1 | model | StrOutputParser()

resultado = chain_simples.invoke({"texto": "Python é uma linguagem de programação"})
print("Chain simples:", resultado)

Chain simples: Python é uma linguagem de programação de alto nível, versátil e de fácil leitura.


In [76]:
# === EXEMPLO 2: Com RunnablePassthrough - mantendo dados originais ===
from langchain_core.runnables import RunnableParallel

prompt2 = ChatPromptTemplate.from_template("Analise: {texto}")

# Chain paralela: processa E mantém original
chain_paralela = RunnableParallel(
    {
        "original": RunnablePassthrough(),  # Mantém dados originais
        "analise": prompt2 | model | StrOutputParser(),  # Processa dados
    }
)

resultado = chain_paralela.invoke({"texto": "LangChain é poderoso"})
print("Original:", resultado["original"])
print("Análise:", resultado["analise"])

Original: {'texto': 'LangChain é poderoso'}
Análise: LangChain é uma biblioteca projetada para facilitar o desenvolvimento de aplicações que utilizam modelos de linguagem, especialmente no contexto de integração com sistemas e fluxos de trabalho mais complexos. Aqui estão alguns pontos que destacam o poder do LangChain:

1. **Integração com Várias Fontes de Dados**: LangChain permite que você conecte modelos de linguagem a diferentes fontes de dados, sejam bancos de dados, APIs ou outros sistemas, facilitando a recuperação e o processamento de informações.

2. **Construção de Cadências de Conversa**: A biblioteca oferece suporte para criar diálogos dinâmicos, permitindo que os desenvolvedores criem fluxos de conversa mais naturais e interativos.

3. **Gerenciamento de Estado**: LangChain possibilita gerenciar o estado da conversa, o que é crucial para aplicações que dependem de contexto para fornecer respostas mais precisas e relevantes.

4. **Facilidade de Uso**: A abstração de muitos

In [77]:
# === EXEMPLO 3: Fluxo complexo com contexto ===
def formatar_contexto(data):
    return f"Contexto: {data['contexto']}\nPergunta: {data['pergunta']}"


prompt3 = ChatPromptTemplate.from_template("{texto_formatado}")

chain_complexa = RunnableParallel(
    {
        "dados_originais": RunnablePassthrough(),  # Preserva tudo
        "resposta": {
            "texto_formatado": formatar_contexto  # Transforma dados
        }
        | prompt3
        | model
        | StrOutputParser(),
    }
)

entrada = {
    "contexto": "Python é uma linguagem interpretada",
    "pergunta": "Quais são suas vantagens?",
}

resultado = chain_complexa.invoke(entrada)
print("Dados preservados:", resultado["dados_originais"])
print("Resposta gerada:", resultado["resposta"])

Dados preservados: {'contexto': 'Python é uma linguagem interpretada', 'pergunta': 'Quais são suas vantagens?'}
Resposta gerada: Python, como uma linguagem interpretada, apresenta várias vantagens que a tornam popular entre desenvolvedores. Aqui estão algumas das principais:

1. **Facilidade de Uso**: Python possui uma sintaxe simples e clara, o que facilita o aprendizado e a escrita do código, especialmente para iniciantes.

2. **Portabilidade**: Como uma linguagem interpretada, o código Python pode ser executado em diferentes plataformas (Windows, macOS, Linux) sem necessidade de recompilação, desde que o interpretador esteja disponível.

3. **Interatividade**: Python permite execução de código em tempo real através de ambientes interativos como o interpretador do Python e Jupyter Notebooks, facilitando protótipos rápidos e aprendizado.

4. **Bibliotecas e Frameworks**: Python possui uma vasta gama de bibliotecas e frameworks (como NumPy, Pandas, Flask e Django), que simplificam o de

# === EXEMPLO 4: RAG com contexto preservado ===
chain_rag = RunnableParallel({
    "pergunta_original": RunnablePassthrough(),
    "contexto_recuperado": retriever,  # busca documentos
    "resposta": {
        "context": retriever,
        "question": RunnablePassthrough()
    } | prompt_rag | model | StrOutputParser()
})

# === EXEMPLO 5: Análise multi-etapa ===
chain_analise = RunnableParallel({
    "entrada": RunnablePassthrough(),
    "sentimento": prompt_sentimento | model | StrOutputParser(),
    "resumo": prompt_resumo | model | StrOutputParser(),
    "palavras_chave": prompt_keywords | model | StrOutputParser()
})


![alt text](image-1.png)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# --- Configuração Inicial ---
# Usaremos um modelo de linguagem da OpenAI.
# Lembre-se de configurar sua chave de API da OpenAI no ambiente.
model = ChatOpenAI(model="gpt-3.5-turbo")
parser = StrOutputParser()

# --- 1. Chain para criar o poema ---
prompt_poema = ChatPromptTemplate.from_template(
    "Escreva um poema curto sobre um(a) {animal}."
)
chain_poema = prompt_poema | model | parser

# --- 2. Chain para explicar a inspiração ---
# Note que esta chain precisa tanto do {animal} original quanto do {poema} gerado.
prompt_explicacao = ChatPromptTemplate.from_template(
    "Excelente poema! Qual foi sua inspiração para criar este poema sobre um(a) {animal}?\n\nPoema:\n{poema}"
)
chain_explicacao = prompt_explicacao | model | parser

# --- 3. Combinando tudo com RunnablePassthrough ---
# Aqui está a mágica!
# Criamos um "dicionário" de entradas para a 'chain_explicacao'.
# - 'animal': vem diretamente da entrada original do usuário. Usamos o RunnablePassthrough para isso.
# - 'poema': é o resultado da 'chain_poema'.
chain_completa = (
    {
        "animal": RunnablePassthrough(),  # Pega a entrada original ({'animal': 'gato'}) e passa adiante.
        "poema": chain_poema,  # Executa a primeira chain e coloca o resultado aqui.
    }
    | chain_explicacao
)

# --- Execução ---
# A entrada inicial é um dicionário simples.
entrada = {"animal": "gato"}
resultado = chain_completa.invoke(entrada)

print("--- Animal Solicitado ---")
print(entrada["animal"])
print("\n--- Resultado Final (Explicação e Poema Implícito) ---")
print(resultado)

--- Animal Solicitado ---
gato

--- Resultado Final (Explicação e Poema Implícito) ---


Minha inspiração para criar este poema sobre um gato veio da própria natureza e personalidade desses animais misteriosos e intrigantes. Eles são seres que transmitem uma sensação de liberdade e sabedoria, ao mesmo tempo em que são carinhosos e cativantes. A forma como se movem, como se escondem e como nos conquistam com seu ronronar suave são elementos que me motivaram a escrever sobre eles. Espero que o poema tenha conseguido transmitir um pouco dessa magia e encanto que os gatos possuem.


E se você quisesse fazer duas ou mais coisas ao mesmo tempo com a mesma entrada e depois juntar os resultados? Em vez de rodar uma chain depois da outra, você pode executá-las em paralelo. Isso é mais eficiente e organiza melhor seu código.

O RunnableParallel funciona como um maestro: ele dá a mesma partitura (a entrada) para vários músicos (as chains) e pede que toquem ao mesmo tempo, depois junta tudo em uma bela sinfonia (o resultado final).

Quando usar? Quando você precisa gerar diferentes informações (não dependentes entre si) a partir da mesma entrada.

Exemplo Prático: Análise de um Tópico
Vamos supor que, para um determinado tópico, queremos que a IA gere simultaneamente:
a) Uma piada sobre o tópico.
b) Um fato interessante sobre o mesmo tópico.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

# --- Configuração Inicial ---
model = ChatOpenAI(model="gpt-3.5-turbo")
parser = StrOutputParser()

# --- Chain para a piada ---
prompt_piada = ChatPromptTemplate.from_template("Conte uma piada curta sobre {topico}.")
chain_piada = prompt_piada | model | parser

# --- Chain para o fato interessante ---
prompt_fato = ChatPromptTemplate.from_template(
    "Mencione um fato interessante sobre {topico}."
)
chain_fato = prompt_fato | model | parser

# --- Combinando em Paralelo ---
# Criamos um dicionário onde cada chave corresponde a uma chain.
# O RunnableParallel executa todas elas com a mesma entrada.
mapa_paralelo = RunnableParallel(
    piada=chain_piada,
    fato=chain_fato,
)

# --- Execução ---
entrada = {"topico": "café"}
resultado = mapa_paralelo.invoke(entrada)

print(f"--- Análise sobre: {entrada['topico']} ---")
print("\n--- Piada Gerada ---")
print(resultado["piada"])
print("\n--- Fato Gerado ---")
print(resultado["fato"])

# Você pode até mesmo encadear mais uma chain depois!
prompt_final = ChatPromptTemplate.from_template(
    "Use o seguinte fato para criar um slogan para uma cafeteria:\n\nFATO: {fato}"
)
chain_slogan = prompt_final | model | parser

# Combinando a execução paralela com uma chain sequencial
chain_completa = mapa_paralelo | {"fato": lambda x: x["fato"]} | chain_slogan

slogan_resultado = chain_completa.invoke(entrada)
print("\n--- Slogan Criado a partir do Fato ---")
print(slogan_resultado)

--- Análise sobre: café ---

--- Piada Gerada ---
Por que o café foi ao médico?

Porque ele estava espresso!

--- Fato Gerado ---
O café é a segunda bebida mais consumida no mundo, perdendo apenas para a água.

--- Slogan Criado a partir do Fato ---
Slogan: "Na nossa cafeteria, servimos a segunda melhor bebida do mundo: café!"


RunnableLambda: A Ferramenta de "Faça Você Mesmo"
Às vezes, você precisa fazer uma pequena transformação nos dados entre duas etapas de uma chain. Pode ser algo simples como formatar um texto, pegar um item de uma lista ou extrair uma parte de um dicionário.

Para essas pequenas funções personalizadas, você não precisa criar um componente complexo. Você pode usar o RunnableLambda, que transforma qualquer função Python simples em um componente da sua chain.

Quando usar? Para manipulações de dados rápidas e customizadas entre os passos da chain.

Exemplo Prático: Processando uma Lista de Itens
Vamos pedir à IA para listar 3 filmes sobre um gênero. O modelo vai retornar uma string única com os filmes. Queremos, então, pegar essa string, dividi-la em uma lista e selecionar apenas o segundo filme para pedir uma sinopse.

In [79]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

# --- Configuração Inicial ---
model = ChatOpenAI(model="gpt-3.5-turbo")
parser = StrOutputParser()

# --- Funções Python que usaremos com RunnableLambda ---


# Função para pegar a string do modelo e dividi-la em uma lista de filmes.
# Ex: "1. De Volta para o Futuro\n2. O Exterminador do Futuro\n3. Blade Runner" -> ["De Volta...", "O Exterminador...", ...]
def extrair_lista_de_filmes(texto_do_modelo: str) -> list[str]:
    # Remove os números e o espaço inicial de cada linha
    filmes = [linha.split(". ", 1)[1] for linha in texto_do_modelo.strip().split("\n")]
    return filmes


# Função para pegar o segundo filme da lista.
def pegar_segundo_item(lista: list) -> str:
    # Adicionamos uma verificação para evitar erros se a lista for curta
    return lista[1] if len(lista) > 1 else "Nenhum filme encontrado"


# --- Construindo a Chain ---

# 1. Chain para listar os filmes
prompt_listar = ChatPromptTemplate.from_template(
    "Liste 3 filmes clássicos do gênero {genero}."
)
chain_listar_filmes = prompt_listar | model | parser

# 2. Chain para obter a sinopse de um filme específico
prompt_sinopse = ChatPromptTemplate.from_template(
    "Qual é a sinopse do filme '{filme}'?"
)
chain_sinopse = prompt_sinopse | model | parser

# 3. Combinando tudo com RunnableLambda
# O fluxo é:
# Listar Filmes -> Extrair a Lista -> Pegar o Segundo Filme -> Pedir a Sinopse
chain_completa = (
    chain_listar_filmes
    | RunnableLambda(extrair_lista_de_filmes)
    | RunnableLambda(pegar_segundo_item)
    | chain_sinopse  # A entrada para esta chain é a saída do RunnableLambda anterior
)

# --- Execução ---
entrada = {"genero": "Ficção Científica"}
resultado = chain_completa.invoke(entrada)

print(f"--- Buscando sinopse para o segundo filme do gênero: {entrada['genero']} ---")
print("\n--- Resultado Final (Sinopse) ---")
print(resultado)

--- Buscando sinopse para o segundo filme do gênero: Ficção Científica ---

--- Resultado Final (Sinopse) ---
No ano de 2019, uma corporação desenvolve uma linha de clones humanos chamados de replicantes, utilizados para trabalhos perigosos em colônias fora da Terra. Por lei, esses replicantes têm uma vida útil de apenas quatro anos e, quando alguns deles se rebelam e voltam para a Terra em busca de mais tempo de vida, entra em ação Rick Deckard, um ex-policial contratado como caçador de replicantes. Em seu trabalho, Deckard terá que enfrentar desafios morais e éticos enquanto tenta capturar os fugitivos, incluindo o misterioso e enigmático Roy Batty. Ao longo da trama, Deckard começa a questionar sua própria humanidade e a dos replicantes, levantando questões existenciais sobre o que realmente significa ser humano.


In [80]:
from langchain_core.runnables import RunnableLambda


# Definindo uma função simples
def cumprimentar(nome):
    return f"Olá, {nome}!"


# Criando um RunnableLambda a partir da função
runnable_cumprimentar = RunnableLambda(cumprimentar)
# Invocando o RunnableLambda
resultado = runnable_cumprimentar.invoke("Maria")
print(resultado)

Olá, Maria!


In [82]:
from langchain_core.runnables import RunnableLambda, RunnableParallel


# Funções que recebem dicionários (entrada do RunnableParallel)
def adicionar_um(data):
    return data["x"] + 1


def multiplicar_por_dois(data):
    return data["x"] * 2


# Criando Runnables
runnable_adicionar = RunnableLambda(adicionar_um)
runnable_multiplicar = RunnableLambda(multiplicar_por_dois)

# RunnableParallel com dicionário
runnable_paralelo = RunnableParallel(
    {"adicionar": runnable_adicionar, "multiplicar": runnable_multiplicar}
)

# Executando
resultado = runnable_paralelo.invoke({"x": 3})
print(resultado)  # {'adicionar': 4, 'multiplicar': 6}

{'adicionar': 4, 'multiplicar': 6}


### Pipeline de processamento de linguagem

Este padrão de "traduzir e depois resumir" é extremamente comum e poderoso. Dominar essa técnica de conectar chains, adaptando as entradas e saídas, é o segredo para construir aplicações complexas com LangChain.

### Análise do Tutor: O que Aconteceu?

1.  **Entrada**: A `chain_combinada` recebeu `{"eng_text": "Artificial Intelligence agents..."}`.
2.  **Primeiro Passo**: A `chain_pt` foi executada. Ela pegou o texto em inglês, traduziu e sua saída foi a string: `"Agentes de Inteligência Artificial são entidades autônomas..."`.
3.  **O "Adaptador"**: Essa string foi passada para o `RunnableLambda`. A função `lambda texto_traduzido: {"texto": texto_traduzido}` foi executada, transformando a string no dicionário `{"texto": "Agentes de Inteligência Artificial são entidades autônomas..."}`.
4.  **Segundo Passo**: Esse dicionário foi então passado para a `chain_resumo`. Como o formato estava perfeito, ela conseguiu extrair o valor da chave `texto`, colocá-lo em seu prompt e gerar o resumo final.
5.  **Saída Final**: O resultado de todo o processo é a string com o resumo: "Entidades autônomas que percebem ambiente." (ou algo similar, dependendo da execução do modelo).

Você acaba de criar uma pipeline de processamento de linguagem\! Este padrão de "traduzir e depois resumir" é extremamente comum e poderoso. Dominar essa técnica de conectar chains, adaptando as entradas e saídas, é o segredo para construir aplicações complexas com LangChain.